In [1]:
import torch
import torchvision
import torch.optim as optim
from data.data import get_train_test_loader
from model.network import FastUpdateNet, FastUpdateNetLarge, FastUpdateNetLarge_Better, FastUpdateNetLarge_Best
import torch.nn.functional as F

import pickle
import time 

/home/kyle/pythonenvs/venv38/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
n_epochs = 5
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
# torch.manual_seed(random_seed)

In [3]:
train_loader, test_loader = get_train_test_loader('mnist')

In [4]:
train_losses = []
train_counter = []
test_losses = []
test_accuracies = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [5]:
from threading import Thread

def compute_back_M1(network):
  network.mNet1.backwardHidden()

def compute_back_M2(network):
  network.mNet2.backwardHidden()

def train(epoch, network):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data= data.to('cuda:1')
    target = target.to('cuda:1')
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    # print(network.mNet.saver.grad)
    # p1 = Thread(target=compute_back_M1, args=[network]) # start two independent threads
    # p2 = Thread(target=compute_back_M2, args=[network])
    
    # p1.start()
    # p2.start()
    network.mNet1.backwardHidden()
    network.mNet2.backwardHidden()
    # p1.join() # wait for the two threads to finish
    # return 
    # p2.join()

    correctness = (target == torch.argmax(output))
    optimizer.step()
    # network.mNet.weightUpdate(correctness, lr = learning_rate)
    if batch_idx % log_interval == 0:
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))


In [6]:
def test(network):
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data = data.to('cuda:1')
      target = target.to('cuda:1')
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  test_accuracies.append(100. * correct / len(test_loader.dataset))
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

## Test using 2 Ms (computing grad in main thread and separate thread)

In [7]:

import torch.nn as nn
fNet = FastUpdateNetLarge_Best().to('cuda:1')
# fNet.mNet1 = torch.nn.Sequential(nn.Linear(392, 196), nn.ReLU(), nn.Linear(196, 98), nn.ReLU(), nn.Linear(98, 49), nn.ReLU())
# fNet.mNet2 = torch.nn.Sequential(nn.Linear(392, 196), nn.ReLU(), nn.Linear(196, 98), nn.ReLU(), nn.Linear(98, 49), nn.ReLU())
optimizer = optim.Adam(fNet.parameters(), lr=learning_rate,)# momentum=momentum)

# optimizer = optim.SGD(fNet.parameters(), lr=learning_rate, momentum=momentum)

In [8]:
train_losses = []
train_counter = []
test_losses = []
test_accuracies = []

s = time.perf_counter()
# print(fNet)

test(fNet)
for epoch in range(1, n_epochs + 1):
  train(epoch, fNet)
  test(fNet)
  # if (epoch + 1) % 10 == 0:
  #   torch.save(fNet, 'fNet-stdp-30000-epoch.pt')
  #   with open('fNet-stdp-3000-epoch-loss-accuracy.pkl', 'wb') as f:
  #     pickle.dump({'train_losses':train_losses, 'test_losses': test_losses, 'test_accuracies':test_accuracies}, f, protocol=pickle.HIGHEST_PROTOCOL)

# torch.save(fNet, 'fNet-stdp-30000-epoch.pt')
e = time.perf_counter()
print(f"Training time for {epoch} epochs: {e-s} seconds")

/home/kyle/projects/cs674/magic-m/model/network.py:308: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(o_5)
/home/kyle/pythonenvs/venv38/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3062, Accuracy: 1326/10000 (13%)


Test set: Avg. loss: 0.3431, Accuracy: 8985/10000 (90%)


Test set: Avg. loss: 0.2831, Accuracy: 9109/10000 (91%)


Test set: Avg. loss: 0.7656, Accuracy: 7949/10000 (79%)


Test set: Avg. loss: 0.4797, Accuracy: 8658/10000 (87%)


Test set: Avg. loss: 0.3999, Accuracy: 8884/10000 (89%)

Training time for 5 epochs: 55.90331888431683 seconds


In [9]:
test_accuracies

[tensor(13.2600, device='cuda:1'),
 tensor(89.8500, device='cuda:1'),
 tensor(91.0900, device='cuda:1'),
 tensor(79.4900, device='cuda:1'),
 tensor(86.5800, device='cuda:1'),
 tensor(88.8400, device='cuda:1')]

## Test using no Ms, and only on main thread

In [10]:
def train_no_M(epoch, network):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data = data.to('cuda:1')
    target = target.to('cuda:1')
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()

    correctness = (target == torch.argmax(output))
    optimizer.step()
    # network.mNet.weightUpdate(correctness, lr = learning_rate)
    if batch_idx % log_interval == 0:
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [11]:

import torch.nn as nn
fNet = FastUpdateNetLarge_Best(use_M=False).to('cuda:1')

optimizer = optim.Adam(fNet.parameters(), lr=learning_rate,)# momentum=momentum)

# optimizer = optim.SGD(fNet.parameters(), lr=learning_rate, momentum=momentum)

In [12]:
train_losses = []
train_counter = []
test_losses = []
test_accuracies = []

s = time.perf_counter()

test(fNet)
for epoch in range(1, n_epochs + 1):
  train_no_M(epoch, fNet)
  test(fNet)
  # if (epoch + 1) % 10 == 0:
  #   torch.save(fNet, 'fNet-stdp-30000-epoch.pt')
  #   with open('fNet-stdp-3000-epoch-loss-accuracy.pkl', 'wb') as f:
  #     pickle.dump({'train_losses':train_losses, 'test_losses': test_losses, 'test_accuracies':test_accuracies}, f, protocol=pickle.HIGHEST_PROTOCOL)

# torch.save(fNet, 'fNet-stdp-30000-epoch.pt')
e = time.perf_counter()
print(f"Training time for {epoch} epochs: {e-s} seconds")


Test set: Avg. loss: 2.3093, Accuracy: 981/10000 (10%)


Test set: Avg. loss: 0.2258, Accuracy: 9429/10000 (94%)


Test set: Avg. loss: 0.2098, Accuracy: 9488/10000 (95%)


Test set: Avg. loss: 0.1996, Accuracy: 9416/10000 (94%)


Test set: Avg. loss: 0.2540, Accuracy: 9508/10000 (95%)


Test set: Avg. loss: 0.2797, Accuracy: 9524/10000 (95%)

Training time for 5 epochs: 51.2263676491566 seconds
